# Decision Trees from scratch in Python

## VIDEO :

In [1]:
# Decision Trees

from IPython.display import IFrame
url1 = 'https://drive.google.com/file/d/'
url2 = '/preview'
id = '1O5eaH-H8M83v3JRe3BqQv8BX2a1h3A8a'
url =url1+id+url2
IFrame(url,width="600",height="400")

In [2]:
# Decision Trees

from IPython.display import IFrame
url1 = 'https://drive.google.com/file/d/'
url2 = '/preview'
id = '15sP1Y4gQXO4GH8NzOJSB5Bp1KK5RsZ71'
url =url1+id+url2
IFrame(url,width="600",height="400")

## Theory

Refer the following pdf: [DecisionTrees](https://drive.google.com/open?id=1A5B6e4zOVCMk9QH6FB6moDHbyQ0vZZ-g)

## Implementation

We shall use the golf game example given in pdf. Since the dataset is small we shall create it here itself.We have just looked at Mathematical working for ID3, this post we will see how to build this in Python from the scratch. We will make it simple by using Pandas dataframes.

In [3]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
# ‘eps’ here is the smallest representable number. At times we get log(0) or 0 in the 
# denominator, to avoid that we are going to use this.
from numpy import log2 as log

In [4]:
# Dataset

dataset = {
    'outlook':['Sunny','Sunny','Overcast','Rain','Rain','Rain','Overcast','Sunny','Sunny','Rain','Sunny','Overcast','Overcast','Rain'],
    'temp':['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild'],
    'humidity':['High','High','High','High','Normal','Normal','Normal','High','Normal','Normal','Normal','High','Normal','High'],
    'wind':['Weak','Strong','Weak','Weak','Weak','Strong','Weak','Weak','Weak','Strong','Strong','Strong','Weak','Strong'],
    'play_tennis':['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']
}

In [5]:
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','wind','play_tennis'])

In [6]:
len(dataset['outlook'])==len(dataset['temp'])==len(dataset['humidity'])==len(dataset['wind'])==len(dataset['play_tennis'])

True

In [7]:
df

,outlook,temp,humidity,wind,play_tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Weak,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Strong,Yes


We have seen from an earlier post we need to find the Entropy and then Information Gain for splitting the data set.

![entropy](img/entropy.png)

We’ll define a function that takes in class (target variable vector) and finds the entropy of that class.

Here the fraction is ‘pi’, it is the proportion of a number of elements in that split group to the number of elements in the group before splitting(parent group).

In [8]:
def find_entropy_key(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [9]:
# Entropy of predictive class here ['Yes', 'No']
find_entropy_key(df)

0.9402859586706309

In [10]:
def find_entropy_attribute(df,attribute):
    
    Class = df.keys()[-1]   #Since the last attrib is out predictive class
    
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
   
    entropy2 = 0
    
    # Consider unique variables in attribute we passed
    for variable in variables:
        
        entropy = 0
        
        # Target classes of predictive class here only Yes and No
        for target_variable in target_variables:
            # num e.g can be thought as
            # df['temp'][df['temp']=='Hot'][df['play_tennis'] =='Yes']
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            
            # den can be thought as
            # df['temp'][df['temp']=='Hot']
            den = len(df[attribute][df[attribute]==variable])
            
            fraction = num/(den+eps)
            
            entropy += -fraction*log(fraction+eps)
            
        # Weighted avg entropy of childern   
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
        
    return abs(entropy2)

In [11]:
np.argmax?

In [12]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy_key(df)-find_entropy_attribute(df,key))
        
    return df.keys()[:-1][np.argmax(IG)]

In [13]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

In [14]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['play_tennis'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [15]:
# Now we build the tree
tree = buildTree(df)

In [16]:
import pprint
pprint.pprint(tree)

{'outlook': {'Overcast': 'Yes',
             'Rain': {'wind': {'Strong': {'temp': {'Cool': 'No',
                                                   'Mild': {'humidity': {'High': 'No',
                                                                         'Normal': 'Yes'}}}},
                               'Weak': 'Yes'}},
             'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [17]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [18]:
instance = df.iloc[6]
print(instance)

outlook        Overcast
temp               Cool
humidity         Normal
wind               Weak
play_tennis         Yes
Name: 6, dtype: object


In [19]:
predict(instance, tree)

'Yes'

In [20]:
# New data
data = {'outlook': 'Rain', 'temp':'Mild', 'humidity':'Normal', 'wind':'Strong'}
inst = pd.Series(data)

In [21]:
predict(inst, tree)

'Yes'

That's all for this notebook. Special Thanks to: [Rakend Dubba](https://medium.com/@rakendd)